In [ ]:
import pandas as pd
from db_utils import Plotter, DataTransform
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
df = pd.read_csv('data2.csv')
'''get back to these
#PL = Plotter()
#DT = DataTransform()
#columns = ['open_accounts','total_accounts','out_prncp','out_prncp_inv','annual_inc','loan_amount','funded_amount','funded_amount_inv','dti','instalment','int_rate','delinq_2yrs','inq_last_6mths']
'''
'''useless
#df = PL.generate_a_plot_for_nulls(df)

#df = PL.generate_a_plot_for_nulls(df)
#print(df.nunique())

#make a scatter plot
#print(df.describe())

#for i in df:
#    print(df.describe()[[i]])

#plt.scatter(df.index, df['funded_amount'])
#plt.scatter(df.index, df['funded_amount'])
#plt.scatter(df.index, df['funded_amount'])
#plt.scatter(df.index, df['funded_amount'])
#plt.scatter(df.index, df['funded_amount'])
'''
'''describe
print(df.describe()[['loan_amount']])
print(df.describe()[['funded_amount']])
print(df.describe()[['funded_amount_inv']])
print(df.describe()[['int_rate']])
print(df.describe()[['instalment']])
#print(df.describe()[['grade']])
#print(df.describe()[['sub_grade']])
#print(df.describe()[['employment_length']])
#print(df.describe()[['home_ownership']])
print(df.describe()[['annual_inc']])
#print(df.describe()[['verification_status']])
#print(df.describe()[['issue_date']])
#print(df.describe()[['loan_status']])
#print(df.describe()[['payment_plan']])
#print(df.describe()[['purpose']])
print(df.describe()[['dti']])
print(df.describe()[['delinq_2yrs']])
#print(df.describe()[['earliest_credit_line']])
print(df.describe()[['inq_last_6mths']])
print(df.describe()[['open_accounts']])
print(df.describe()[['total_accounts']])
print(df.describe()[['out_prncp']])
print(df.describe()[['out_prncp_inv']])['loan_amount']
'''
columns = ['open_accounts','total_accounts','out_prncp','out_prncp_inv', 'annual_inc', 'loan_amount', 'funded_amount', 'funded_amount_inv', 'dti', 'instalment', 'int_rate','delinq_2yrs', 'inq_last_6mths']

#this is now in db_utils
def plot_boxplots(df, columns):
    plt.figure(figsize=(15, 8))
    
    for i, col in enumerate(columns, 1):
        plt.subplot(1, len(columns), i)
        sns.boxplot(y=df[col])
        plt.title(f'Boxplot of {col}')
        plt.ylabel(col)
    plt.tight_layout()
    plt.show()
#this is now in db_utils
print('original boxplots')
plot_boxplots(df, columns) #prints out boxplots of the original dataframe, df


In [ ]:
#Z-Score Method - VIDEO VERSION
# trimming - delete the outlier data - from video
def z_score_trim_vidver(df, columns):
    for i in df[(columns)]:
        upper_limit = df[i].mean() + 3*df[i].std()
        lower_limit = df[i].mean() - 3*df[i].std()
        df_z_score_vid_trim = df.loc[(df[i]<upper_limit) & (df[i]>lower_limit)]
    print(len(df)-len(df_z_score_vid_trim), 'outliers removed using zscore with vid trimming method')
    #plot_boxplots(df_z_score_vid_trim, columns)
    #return df_z_score_vid_trim
print(z_score_trim_vidver(df, columns))

# capping - change the outlier values to upper (or) lower limit values - from video
def z_score_cap_vidver(df, columns):
    for i in df[(columns)]:
        upper_limit = df[i].mean() + 3*df[i].std()
        lower_limit = df[i].mean() - 3*df[i].std()
        df_z_score_vid_capping = df.copy()
        df_z_score_vid_capping.loc[df_z_score_vid_capping[i]>upper_limit, i] = upper_limit
        df_z_score_vid_capping.loc[df_z_score_vid_capping[i]<lower_limit, i] = lower_limit
    print(len(df)-len(df_z_score_vid_capping), 'outliers removed using zscore with vid capping method')
    #plot_boxplots(df_z_score_vid_capping, columns)
    #return df_z_score_vid_capping
print(z_score_cap_vidver(df, columns))
#actually, len-len should be zero as they are now within the limits?
# number of at limit items should increase but I dont know how to express that

In [ ]:
#Z-Score Method - NOTEBOOK VERSION

# Calculate the Z-Scores for YoE
def create_z_score_df(df, columns):
    df_with_zscore = df.copy() # new dataframe is a copy of old dataframe
    for i in df_with_zscore[(columns)]:
        #print(i)
        #print(np.mean(df_with_zscore[i]))
        mean_of_col = np.mean(df_with_zscore[i])
        std_of_col = np.std(df_with_zscore[i])
        z_scores = ((df_with_zscore[i] - mean_of_col) / std_of_col)
    df_with_zscore['z_scores'] = z_scores
    return df_with_zscore
df_with_zscore = create_z_score_df(df, columns)

#trimming
def trim_by_z_score(df_with_zscore):
    df_trimmed_by_z_score = df_with_zscore.loc[(df_with_zscore['z_scores']> -3) & (df_with_zscore['z_scores']<3)]
    print('Number of outliers trimmed: ', (len(df_with_zscore)-len(df_trimmed_by_z_score)))
trim_by_z_score(df_with_zscore)

#capping
#before and after demo
def cap_by_z_score(df_with_zscore, columns):
    def replace_by_z_score(dataframe, df_column, threshold=3): #caps a df
        mean = dataframe[df_column].mean()
        std = dataframe[df_column].std()
        # getting the outliers rows
        outliers = dataframe.loc[((dataframe[df_column] - mean) / std).abs() > threshold, df_column]
        # replacing the outlier rows with the mean of the column
        dataframe.loc[outliers.index, df_column] = mean
        return dataframe
    df_capped_by_z_score = df_with_zscore.copy()
    #print('Before Capping:\n', df_capped_by_z_score[(columns)].skew())
    #plot_boxplots(df_capped_by_z_score, columns)
    for i in df_capped_by_z_score[(columns)]:
        replace_by_z_score(df_capped_by_z_score, i)
    #print('After Capping:\n', (df_capped_by_z_score[(columns)].skew()))
    #plot_boxplots(df_capped_by_z_score, columns)
    print('Number of Capped outliers:', (len(df)-len(df_capped_by_z_score)))
cap_by_z_score(df_with_zscore, columns)



In [ ]:
# IQR Method - from video, same as from notebook
def get_upper_limit(df, columns):
    for i in df[(columns)]:
        q1 = df[i].quantile(0.25)
        q3 = df[i].quantile(0.75)
    IQR = q3-q1
    upper_limit = q3 + (1.5 * IQR)
    return upper_limit
def get_lower_limit(df, columns):
    for i in df[(columns)]:
        q1 = df[i].quantile(0.25)
        q3 = df[i].quantile(0.75)
    IQR = q3-q1
    lower_limit = q1 - (1.5 * IQR)
    return lower_limit

uplim = get_upper_limit(df, columns)
lowlim = get_lower_limit(df, columns)

print(uplim, lowlim)

# trimming
def trim_by_iqr_limits(df, columns, upper_limit, lower_limit):
    df_untrimmed_by_iqr = df.copy()
    for i in df_untrimmed_by_iqr[(columns)]:
        df_trimmed_by_iqr = df_untrimmed_by_iqr.loc[(df_untrimmed_by_iqr[i] > lower_limit) & (df_untrimmed_by_iqr[i] < upper_limit)]
    print('Number of trimmed outliers:', (len(df_untrimmed_by_iqr) - len(df_trimmed_by_iqr)))
    return df_trimmed_by_iqr
trim_by_iqr_limits(df, columns, uplim, lowlim)

#capping
def cap_by_limits(df, columns, upper_limit, lower_limit):
    df_uncapped = df.copy()
    for i in df[(columns)]:
        df_uncapped.loc[df_uncapped[i]>upper_limit, i] = upper_limit
        df_uncapped.loc[df_uncapped[i]<lower_limit, i] = lower_limit
        outliers = df_uncapped.loc[(df[i] < lower_limit) & (df[i] > upper_limit)]
    df_capped_by_limits = df_uncapped
    print('Number of capped outliers', (len(df)-len(df_capped_by_limits)))
    return df_capped_by_limits
cap_by_limits(df, columns, uplim, lowlim)